# Database Connections through Python

The DOF Property Modeling Team utilizes a range of databases and tools to store and analyze data.

The diagram below gives a high-level overview of how these different data resources and tools are used together.

<img src="resources/data_workflow.jpg"/>

## How can you access these data resources in Python?

There are currently two different database tools that are used: Microsoft SQL Server and Oracle databases. Even though we'll be using the same tools to connect to them, creating the connections with these two different systems requires slightly different syntax and credentials.

### Connecting to Microsoft SQL Server Databases (Production, Test, Sandbox)

We can establish a direct connection to Microsoft SQL Server using existing R packages, which link to the database using our credentials, allowing us to query the Production, Test, and Sandbox databases.

To do this, you first need to import two python packages locally: `python-dotenv` and `pyodbc`. Note: you only need to do this once.

In [ ]:
#install necessary packages

#conda install python-dotenv
#conda install pyodbc

Once these two packages are installed, you can establish a database connection using the credentials for either the production, test, or sandbox databases. For this demonstration, I'll query the test database, so first I'll save those credentials to my local `.Renviron` file saved in my HOME directory (detailed instructions on that [in R](proxy_R.md) and [in python](proxy_python.md)).

```
#replace the following with the real credentials
test_server="XXX.XXX.XX.XXX"
test_database='V8_XXXXXXXXXXXXXXXXXXX'
test_username="XXXXX"
test_password="XXXXX"

production_server="XXX.XXX.XX.XXX"
production_database='V8_XXXXXXXXXXXXXXXXXXX'
production_username="XXXXX"
production_password="XXXXX"

sandbox_server="XXX.XXX.XX.XXX"
sandbox_database='V8_XXXXXXXXXXXXXXXXXXX'
sandbox_username="XXXXX"
sandbox_password="XXXXX"
```

Now that you're credentials are saved locally, restart your kernel (at the top of the screen select the Restart button) and read in your environmental variables. 

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv("C:/Users/BoydClaire/.Renviron"))

True

To check if we can re-read them in successfully, you can try the following:

In [2]:
import os

os.getenv('test_database')

'V8_NEWYORK_NY_TEST_LIVE_NOV2024'

Next, I'll use the pyobdc package to create a connection to the Microsoft SQL Server database as follows:

In [3]:
import pyodbc

# choose which database you want to connect to (assumes you have "{database}_server", "{database}_database", "{database}_username", and "{database}_password" as environmental variables)
database = "test"

# build connection string
connection_args = ["Driver={SQL Server};",
                    f"SERVER={os.environ[f'{database}_server']};",
                    f"DATABASE={os.environ[f'{database}_database']};",
                    f"UID={os.environ[f'{database}_username']};",
                    f"PWD={os.environ[f'{database}_password']};",
                    f"Trusted_Connection=no;",
                    "Port=1433"
]
connection_string = "".join(connection_args)

# create connection object
cnxn = pyodbc.connect(connection_string)

With the database connection in place, we can begin constructing our queries. Let's start with a basic one.

In [5]:
query = "SELECT TOP 100 * FROM REAL_PROP.REALMAST"

Now that we have saved the query as a string, we can use it to query the database and store the result as a dataframe in our local R environment. To do this, we can use the [`read_sql()`](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html) function from pandas, providing the database connection and the query string we wish to execute.

In [6]:
import pandas as pd

# read in data with query and connection
realmast_top100 = pd.read_sql(sql=query, con=cnxn)

In [14]:
# view data
realmast_top100.head()

,REM_MNC,REM_PID,REM_PIN,REM_OWN_NAME,REM_ACCT_NUM,REM_PRCL_LOCN,REM_PRCL_LOCN_STR_PFX,REM_PRCL_LOCN_STREET,REM_PRCL_LOCN_STR_SFX,REM_PRCL_LOCN_NUM,...,REM_PRCL_LOCN_POST_DIRECTION,REM_PRCL_LOCN_PRE_DIRECTION,REM_PRCL_LOCN_STREET_TYPE,REM_PRCL_LOCN_APT_TYPE,REM_USRFLD,REM_USRFLD_DESC,REM_PRCL_LOCN_ADDRESS_ID,REM_BLOCK_SUFFIX,REM_PROPERTY_TYPE,REM_PROPERTY_TYPE_DESC
0,9000.0,1.0,,GOVERNORS ISLAND CORPORATION,1000010010,301 COMFORT COURT,,COMFORT COURT,,301.0,...,None,None,None,None,None,None,None,000,R,None
1,9000.0,2.0,,OWNER NAME,1000010101,1 LIBERTY ISLAND,,LIBERTY ISLAND,,1.0,...,None,None,None,None,None,None,None,000,R,None
2,9000.0,3.0,None,OWNER NAME,1000010201,1 ELLIS ISLAND,None,ELLIS ISLAND,None,1.0,...,None,None,None,None,None,None,None,000,R,None
3,9000.0,4.0,None,OWNER NAME,1000020001,4 SOUTH STREET,None,SOUTH STREET,None,4.0,...,None,None,None,None,None,None,None,000,R,None
4,9000.0,5.0,,OWNER NAME,1000020023,1 PIER 6,,PIER 6,,1.0,...,None,None,None,None,None,None,None,000,R,None


Great! Now let's try a slightly more complicated query using some environmental variables.

[F strings](https://www.geeksforgeeks.org/formatted-string-literals-f-strings-python/) are helpful to drop in variables within a string (like a fiscal year or BCAT) using curly brackets, making it a more legible query than using paste().

In [8]:
prior_FY=2025

incdata_query = \
    f"""
    Select ILA_Pid as Pid,
        ILA_AGI as AGI,
        ILA_AGI_PGSF as Inc_PGSF,
        ILA_Adj_Exp as AdjExp,
        ILA_Adj_Exp_PGSF as Exp_PGSF,
        ILA_Adj_EXP_PGI as ExpRatio,
        ILA_Adj_NOI as NOI,
        ILA_ADJ_NOI_PGSF as NOI_Pgsf,
        ILA_Tot_cap_rate as TotCap,
        ILA_Inc_Val as FY26_MV,
        ILA_Tot_Val as Tot_Val,
        ILF_BLDG_CAT as BCAT,
        ILF_Bldg_Sub_Cat as Subcat
    From 
        Real_Prop.INCOME_LF_APPROACH, 
        Real_Prop.Income_LF
    Where ILA_Type='R' And ILA_Year={prior_FY} 
        and ilf_year={prior_FY} and ilf_pid=ila_pid
    """

We can check if it included our variables correctly by printing it:

In [10]:
print(incdata_query)


    Select ILA_Pid as Pid,
        ILA_AGI as AGI,
        ILA_AGI_PGSF as Inc_PGSF,
        ILA_Adj_Exp as AdjExp,
        ILA_Adj_Exp_PGSF as Exp_PGSF,
        ILA_Adj_EXP_PGI as ExpRatio,
        ILA_Adj_NOI as NOI,
        ILA_ADJ_NOI_PGSF as NOI_Pgsf,
        ILA_Tot_cap_rate as TotCap,
        ILA_Inc_Val as FY26_MV,
        ILA_Tot_Val as Tot_Val,
        ILF_BLDG_CAT as BCAT,
        ILF_Bldg_Sub_Cat as Subcat
    From 
        Real_Prop.INCOME_LF_APPROACH, 
        Real_Prop.Income_LF
    Where ILA_Type='R' And ILA_Year=2025 
        and ilf_year=2025 and ilf_pid=ila_pid
    


Great! Now let's query our test database for this income data.

In [12]:
income = pd.read_sql(sql=incdata_query, con=cnxn)

Let's take a look at the results:

In [13]:
income.head()

,Pid,AGI,Inc_PGSF,AdjExp,Exp_PGSF,ExpRatio,NOI,NOI_Pgsf,TotCap,FY26_MV,Tot_Val,BCAT,Subcat
0,8001.0,0.0,0.00,0.0,0.00,0.0000,0.0,0.00,14.903,0.0,0.0,SP10,1
1,8006.0,2888590.0,37.52,948492.0,12.32,32.8358,1940098.0,25.20,12.546,15463877.0,15464000.0,RU32,1
2,8016.0,285246.0,36.57,46800.0,6.00,16.4069,238446.0,30.57,16.023,1488148.0,1488000.0,K131,M
3,8216.0,0.0,0.00,0.0,0.00,0.0000,0.0,0.00,16.843,0.0,0.0,H001,CA
4,8326.0,61342531.0,53.14,28593423.0,24.77,46.6127,32749108.0,28.37,14.323,228646987.0,228647000.0,O007,1


Now we can analyze the data easily within Python and re-run the same import script without needing to handle any CSVs.

Once you are done with reading in data from the database, close the connection by doing the following:

In [5]:
cnxn.close()

### Connecting to Oracle Databases (FDW, PTS)

This set up is very similar to the above, but we have to pass in different parameters to make a connection to the Oracle databases where the FDW and PTS data is stored. Just like in SAS, we need the database path, schema, username and password to access the database. Before moving forward, add the following to your `.Renviron` file like the following:

```
fdw_username=XXXXXXXXXX
fdw_password=XXXXXXXXXX
fdw_path=XXXXXXXX
fdw_schema=XXXXXXXXXX

pts_username=XXXXXXXXXXXX
pts_password=XXXXXXXXXXX
pts_path=XXXXXXXXXXXXX
pts_schema=XXXXXXXX
```

Restart your kernel and re-read in your new environmental variables. 

In [ ]:
load_dotenv(find_dotenv("C:/Users/BoydClaire/.Renviron"))

To check if we can re-read them in successfully, you can try the following:

In [15]:
os.getenv("fdw_schema")

'fdw3nf'

Using these credentials, we can use the same packages to create a connection to the Oracle database.

In [7]:
# build connection string
database = "fdw"
database_schema = os.getenv(f"{database}_schema")

connection_args = ["Driver={Oracle in OraClient19Home1};",
                    f"DBQ={os.environ[f'{database}_path']};"
                    f"DATABASE={os.environ[f'{database}_schema']};"
                    f"UID={os.environ[f'{database}_username']};",
                    f"PWD={os.environ[f'{database}_password']};",
                    f"Trusted_Connection=no;",
                    "Port=1433"
]
connection_string = "".join(connection_args)

# create connection object
cnxn = pyodbc.connect(connection_string)

To explore what tables are in the database, we can do the following:

In [8]:
cursor = cnxn.cursor()
tables_raw = cursor.tables(schema='FDW3NF')
tables = pd.DataFrame(tables_raw)
tables.columns = ['list']
cleaned_tables = pd.DataFrame(tables['list'].apply(list).to_list(), columns = ['CATALOG','SCHEMA','NAME','TYPE','SELF_REFERENCING_COL_NAME'])

cleaned_tables.head()

This is often helpful when trying to map tables in SQL Server databases back to a table in Oracle.

Next, we can repeat the above steps with this database connection. Note: the syntax is a bit different now because we are using Oracle SQL syntax which is slightly different than Microsoft SQL syntax. We also need to use the database schema when compiling 

In [11]:
fdw_query = f"SELECT * FROM {database_schema}.VW_CAMA_REALMAST WHERE ROWNUM <= 100"

# read in data with query and connection
data = pd.read_sql(sql=fdw_query, con=cnxn)

In [12]:
data

,REM_MNC,REM_PID,REM_PIN,REM_OWN_NAME,REM_ACCT_NUM,REM_PRCL_LOCN,REM_PRCL_LOCN_STR_PFX,REM_PRCL_LOCN_STREET,REM_PRCL_LOCN_STR_SFX,REM_PRCL_LOCN_NUM,...,REM_TAX_ID,REM_USER_ID,REM_USRFLD,REM_USRFLD_DESC,REM_ACCESS_DATE,REM_CREATE_STAMP,REM_CREATE_DATE,REM_LAST_UPDATE,REM_PROPERTY_TYPE,REM_PROPERTY_TYPE_DESC
0,9000.0,455035.0,None,8322 LIU INC,3060180048,8322 5 AVENUE,None,5 AVENUE,None,8322.0,...,None,None,None,None,None,None,2022-06-16 00:59:59,2024-12-11 23:37:18,R,None
1,9000.0,455135.0,None,KUE-JOR NG,3060200042,683 84 STREET,None,84 STREET,None,683.0,...,None,None,None,None,None,None,2022-06-16 00:59:59,2024-12-11 03:17:36,R,None
2,9000.0,455235.0,None,"CHEN, HAO",3060230049,171 85 STREET,None,85 STREET,None,171.0,...,None,None,None,None,None,None,2022-06-16 00:59:59,2024-12-13 00:11:19,R,None
3,9000.0,455335.0,None,RENEE BASILE,3060260018,428 84 STREET,None,84 STREET,None,428.0,...,None,None,None,None,None,None,2022-06-16 00:59:59,2024-12-11 03:17:38,R,None
4,9000.0,455435.0,None,"WEHBEH INVESTMENTS, LLC",3060280008,8405 FT HAMILTON PARKWAY,None,FT HAMILTON PARKWAY,None,8405.0,...,None,None,None,None,None,None,2022-06-16 00:59:59,2024-12-11 23:37:18,R,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9000.0,453165.0,None,"LIVANOS, DESPINA",3059750125,37 82 STREET,None,82 STREET,None,37.0,...,None,None,None,None,None,None,2022-06-16 00:59:59,2024-12-13 00:11:11,R,None
96,9000.0,453265.0,None,"CRUZ, RICHARD",3059770021,152 79 STREET,None,79 STREET,None,152.0,...,None,None,None,None,None,None,2022-06-16 00:59:59,2024-12-11 03:16:36,R,None
97,9000.0,453365.0,None,MURRAY FINGEROTH,3059790035,366 79 STREET,None,79 STREET,None,366.0,...,None,None,None,None,None,None,2022-06-16 00:59:59,2024-12-11 03:16:50,R,None
98,9000.0,453465.0,None,OWNER NAME,3059810024,544 79 STREET,None,79 STREET,None,544.0,...,None,None,None,None,None,None,2022-06-16 00:59:59,2024-12-11 03:16:51,R,None


In [13]:
cnxn.close()